In [ ]:
import re
import pprint
import csv
import pandas as pd
from openpyxl import load_workbook

In [ ]:
def encontrar_texto_entre_parentesis(archivo, num_file, coincidencias):
    with open(archivo, 'r') as f:
        for linea in f:
            for i, coincidencia in enumerate(texto_entre_parentesis(linea)):
                texto = coincidencia.group(1)
                # Reemplazar para dividir tmb por dos puntos
                texto = texto.replace(':', ';')
                # Dividir el texto por el punto y coma
                partes_texto = texto.split(';')  
                for j, parte in enumerate(partes_texto):
                    parte = parte.strip()
                    if contiene_cuatro_digitos_consecutivos(parte) and parte[0].isupper():
                        if parte not in coincidencias:
                            coincidencias[parte] = {
                                "cap_1": "",
                                "cap_2": "",
                                "cap_3": "",
                                "cap_4": "",
                                "orden": f"{i}-{j}",
                                "url": generar_url(parte)
                        } 
                        coincidencias[parte][f"cap_{num_file}"] = "x" # -{num_linea}:{inicio_coincidencia}")
    return coincidencias

def texto_entre_parentesis(linea):
    return re.finditer(r'\(([^)]+)\)', linea)

def contiene_cuatro_digitos_consecutivos(texto):
    # La expresión regular busca al menos cuatro dígitos consecutivos
    return bool(re.search(r'\d{4,}', texto))

def generar_url(cita):
    cita = cita.replace(' ','+')
    cita = cita.replace('&', 'y')
    cita = cita.replace('\\','')
    url = "https://www.google.com/search?q=" + cita
    return url

def remover_tildes(string):
    tildes = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    for tilde, sin_tilde in tildes.items():
        string = string.replace(tilde, sin_tilde)
    return string

In [ ]:
def exportar_citas(diccionario_citas, nombre_archivo, nombre_hoja):
    
    df_citas = pd.DataFrame.from_dict(diccionario_citas, orient='index')

    # resetear el índice para que las citas sean una columna
    df_citas.reset_index(inplace=True)
    df_citas.rename(columns={'index': 'cita'}, inplace=True)
    
    # ordenamos alfabeticamente
    df_citas.sort_values(by = 'cita', inplace=True)

    df_citas['autor'] = df_citas['cita'].str.split(' ').str[0].str.lower().str.replace(',','').apply(remover_tildes)
    df_citas['año'] = df_citas['cita'].str.split(',').str[-1]

    # cargar el archivo Excel existente
    try:
        with pd.ExcelWriter(nombre_archivo, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df_citas.to_excel(writer, sheet_name=nombre_hoja, index=False)
    except FileNotFoundError:
        df_citas.to_excel(nombre_archivo, sheet_name=nombre_hoja, index=False)


In [ ]:
archivos = ['../caps/01_introduccion.tex', '../caps/02_dti.tex', '../caps/03_sandi.tex']
archivos =  [
    # '../caps/citas_txt/Capítulo 1.txt',
    '../caps/draft_3_pre_reemplazar_citas/01_introduccion.tex',
    '../caps/draft_3_pre_reemplazar_citas/02_dti.tex', 
    '../caps/draft_3_pre_reemplazar_citas/03_sandi.tex', 
    '../caps/draft_3_pre_reemplazar_citas/04_discusion.tex'
]
archivo_exportar = 'citas_en_doc.xlsx'

textos_entre_parentesis = {}
for num_file, archivo in enumerate(archivos,  start=1):
    textos_entre_parentesis = encontrar_texto_entre_parentesis(archivo, num_file, textos_entre_parentesis)

exportar_citas(textos_entre_parentesis, archivo_exportar, nombre_hoja='export')

# Check reemplazo citas

In [ ]:

archivos =  [
    '../caps/01_introduccion.tex',
    '../caps/02_dti.tex', 
    '../caps/03_sandi.tex', 
    '../caps/04_discusion.tex'
]
archivo_exportar = 'citas_en_doc_final.xlsx'

textos_entre_parentesis = {}
for num_file, archivo in enumerate(archivos,  start=1):
    textos_entre_parentesis = encontrar_texto_entre_parentesis(archivo, num_file, textos_entre_parentesis)

textos_entre_parentesis